In [ ]:
import cirq
import numpy as np

import cirq_qubitization as cq

from cirq_qubitization.cirq_algos.qubitization_walk_operator_test import get_walk_operator_for_1d_ising_model

Implements Heisenberg-Limited Phase Estimation of the Qubitized Quantum Walks as described in Section-II B. of [Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity](https://arxiv.org/abs/1805.03662)

In [ ]:
def get_resource_state(m: int):
    den = 1 + 2 ** m
    norm = np.sqrt(2 / den)
    return norm * np.sin(np.pi * (1 + np.arange(2**m)) / den)        
    
def phase_estimation(walk: cq.QubitizationWalkOperator, m: int) -> cirq.OP_TREE:
    reflect = walk._reflect
    walk_regs = walk.registers.get_named_qubits()
    reflect_regs = {k:v for k, v in walk_regs.items() if k in reflect.registers}
    
    reflect_controlled = reflect.controlled(control_values=[0])
    walk_controlled = walk.controlled(control_values=[1])
    reflect_op = reflect.on_registers(**reflect_regs)

    m_qubits = [cirq.q(f'm_{i}') for i in range(m)]
    state_prep = cirq.StatePreparationChannel(get_resource_state(m), name='𝜒_m')

    yield state_prep.on(*m_qubits)
    yield walk_controlled.on_registers(**walk_regs, control=m_qubits[0])
    for i in range(1, m):
        yield reflect_controlled.on_registers(control=m_qubits[i], **reflect_regs)
        yield walk.on_registers(**walk_regs)
        walk = walk ** 2
        yield reflect_controlled.on_registers(control=m_qubits[i], **reflect_regs)
        
    yield cirq.qft(*m_qubits, inverse=True)

In [ ]:
num_sites: int = 6
eps: float = 1e-2
m_bits: int = 4

circuit = cirq.Circuit(phase_estimation(get_walk_operator_for_1d_ising_model(num_sites, eps), m=m_bits))
print(circuit)

In [ ]:
num_sites: int = 200
eps: float = 1e-5
m_bits: int = 14

circuit = cirq.Circuit(phase_estimation(get_walk_operator_for_1d_ising_model(num_sites, eps), m=m_bits))
%time cq.t_complexity(circuit[1:-1])